# Лабораторная работа №5

*Кищенко Екатерина  
ПИН-212*

## Задание

>Провести классификацию найденного датасета, методами решающего дерева и случайного леса.

Импорт библиотек

In [1]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

Загрузка датасета

In [2]:
df = pd.read_csv('metal.csv', encoding='utf-8')
df['Mode']= df['Mode'].map({1: True, 0: False}).astype(bool)
print(df.head(3))
print(df.dtypes)

               Spotify ID              Artist IDs Track Name  \
0  19vHgVS1aukRiQWhTqfKnE  1Ffb6ejR6Fe5IamqA5oRUF   DArkSide   
1  5BgnL6gHauuvxe4Ok6W1aC  1Ffb6ejR6Fe5IamqA5oRUF       LosT   
2  0M3adYbGtyRHACP86dey1H  1Ffb6ejR6Fe5IamqA5oRUF     Throne   

          Album Name        Artist Name(s) Release Date  Duration (ms)  \
0           DArkSide  Bring Me The Horizon   10/18/2023         164790   
1               LosT  Bring Me The Horizon     5/4/2023         205368   
2  That's The Spirit  Bring Me The Horizon    9/11/2015         191520   

   Popularity                                Added By              Added At  \
0          74  spotify:user:woth5clwe5p856iw52alp7uek  2023-10-25T13:10:28Z   
1          72  spotify:user:woth5clwe5p856iw52alp7uek  2023-10-25T13:10:28Z   
2          76  spotify:user:woth5clwe5p856iw52alp7uek  2023-10-25T13:10:28Z   

   ... Key  Loudness   Mode  Speechiness  Acousticness  Instrumentalness  \
0  ...   7    -3.018   True       0.1460      0.00035

Удаление ненужных столбцов из датасета

In [3]:
df = df.drop(['Spotify ID', 'Added By', 'Added At', 'Time Signature'], axis=1)

Удаление строк с пропущенными значениями

In [4]:
df.dropna(inplace=True)

Предобработка данных

In [5]:
# Кодирование категориальных признаков
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])
print(df.head(3))

   Artist IDs  Track Name  Album Name  Artist Name(s)  Release Date  \
0         212        2152         631             182           206   
1         212        5399        1647             182          1178   
2         212        9575        2696             182          1740   

   Duration (ms)  Popularity  Genres  Danceability  Energy  Key  Loudness  \
0         164790          74     972         0.387   0.940    7    -3.018   
1         205368          72     972         0.396   0.939    9    -3.155   
2         191520          76     972         0.426   0.888    0    -3.720   

    Mode  Speechiness  Acousticness  Instrumentalness  Liveness  Valence  \
0   True       0.1460      0.000352          0.000003     0.628    0.456   
1  False       0.2270      0.000285          0.000000     0.212    0.390   
2  False       0.0987      0.000455          0.000000     0.306    0.387   

     Tempo  
0  167.910  
1  185.081  
2  144.111  


In [6]:
# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = df.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
df[numeric_features] = scaler.fit_transform(df[numeric_features])
print(df.head(3))

   Artist IDs  Track Name  Album Name  Artist Name(s)  Release Date  \
0   -1.032943   -1.047387   -1.132416       -1.214946     -1.191165   
1   -1.032943   -0.007295   -0.120908       -1.214946      0.549409   
2   -1.032943    1.330378    0.923455       -1.214946      1.555790   

   Duration (ms)  Popularity    Genres  Danceability    Energy       Key  \
0      -0.919780    2.410398  1.890198      0.252167  0.382949  0.498957   
1      -0.548958    2.298142  1.890198      0.317669  0.377046  1.066209   
2      -0.675508    2.522654  1.890198      0.536006  0.076001 -1.486428   

   Loudness   Mode  Speechiness  Acousticness  Instrumentalness  Liveness  \
0  1.034060   True     0.874084     -0.227497         -0.943709  2.487116   
1  0.992878  False     2.215229     -0.227993         -0.943718 -0.051936   
2  0.823040  False     0.090922     -0.226736         -0.943718  0.521792   

    Valence     Tempo  
0  1.007971  1.476925  
1  0.650282  2.060858  
2  0.634023  0.667594  


Разделение данных на обучающий и тестовый наборы

In [7]:
X = df.drop('Mode', axis=1)
Y = df['Mode']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

**Обучение модели методом решающего дерева**

In [8]:
dt_classifier = DecisionTreeClassifier()

Определение сетки параметров для поиска

In [9]:
dt_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

Поиск оптимальных параметров с использованием кросс-валидации

In [10]:
dt_grid_search = GridSearchCV(dt_classifier, dt_param_grid, cv=3)
dt_grid_search.fit(X_train, Y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

Получение лучших параметров

In [11]:
best_dt_parametrs = dt_grid_search.best_params_
print("Лучшие параметры для решающего дерева:", best_dt_parametrs)

Лучшие параметры для решающего дерева: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2}


Обучение модели с лучшими параметрами

In [12]:
best_dt_classifier = DecisionTreeClassifier(**best_dt_parametrs)
best_dt_classifier.fit(X_train, Y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=20)

Предсказание на тестовом наборе данных

In [13]:
Y_pred_dt = best_dt_classifier.predict(X_test)

Оценка модели

In [14]:
print(classification_report(Y_test, Y_pred_dt))

              precision    recall  f1-score   support

       False       0.57      0.57      0.57       991
        True       0.73      0.74      0.74      1601

    accuracy                           0.67      2592
   macro avg       0.65      0.65      0.65      2592
weighted avg       0.67      0.67      0.67      2592



Оценка модели на тестовом наборе данных

In [15]:
dt_accuracy = accuracy_score(Y_test, Y_pred_dt)
print("Точность решающего дерева на тестовом наборе данных:")
print((dt_accuracy * 100).round(2), '%')

Точность решающего дерева на тестовом наборе данных:
67.36 %


**Обучение модели методом случайного леса**

In [16]:
rf_classifier = RandomForestClassifier()

Определение сетки параметров для поиска

In [17]:
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

Поиск оптимальных параметров с использованием кросс-валидации

In [18]:
rf_grid_search = GridSearchCV(rf_classifier, rf_param_grid, cv=3)
rf_grid_search.fit(X_train, Y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]})

Получение лучших параметров

In [19]:
best_rf_params = rf_grid_search.best_params_
print("Лучшие параметры для случайного леса:", best_rf_params)

Лучшие параметры для случайного леса: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


Обучение модели с лучшими параметрами

In [20]:
best_rf_classifier = RandomForestClassifier(**best_rf_params)
best_rf_classifier.fit(X_train, Y_train)

RandomForestClassifier(max_depth=20, n_estimators=300)

Предсказание на тестовом наборе данных

In [21]:
Y_pred_rf = best_rf_classifier.predict(X_test)

Оценка модели

In [22]:
print(classification_report(Y_test, Y_pred_rf))

              precision    recall  f1-score   support

       False       0.72      0.46      0.56       991
        True       0.73      0.89      0.80      1601

    accuracy                           0.73      2592
   macro avg       0.72      0.67      0.68      2592
weighted avg       0.72      0.73      0.71      2592



Оценка модели на тестовом наборе данных

In [25]:
rf_accuracy = accuracy_score(Y_test, Y_pred_rf)
print("Точность случайного леса на тестовом наборе данных:", (rf_accuracy * 100).round(2), '%')

Точность случайного леса на тестовом наборе данных: 72.53 %
